In [ ]:
import os
import sys
import numpy as np
import neuroglancer
from skimage import io
import matplotlib
from pprint import pprint
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2
from pathlib import Path
from tifffile import imwrite
PIPELINE_ROOT = Path('.').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())

from library.utilities.utilities_mask import normalize16

In [ ]:
animal = 'DK52'
filename = 'princeton_annotations.tif'
DATA = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
filepath = os.path.join(DATA, filename)
volume = io.imread(filepath)
#volume = volume.astype(np.uint32)
print(f'Volume shape={volume.shape} dtype={volume.dtype}')

In [ ]:
sagittal = volume.swapaxes(-2,-1)[...,::-1]
sagittal = np.flip(sagittal, axis=2)
outpath = os.path.join(DATA, 'princeton_annotations.tif')
print(sagittal.shape)
imwrite(outpath, sagittal)

In [ ]:
vm2 = np.swapaxes(volume, 0,2)
outpath = os.path.join(DATA, 'test_princeton_annotations.tif')
imwrite(outpath, vm2)

In [ ]:
sagittal = volume.swapaxes(-2,-1)[...,::-1]
sagittal = np.flip(sagittal, axis=2)
arr = sagittal
#arr = np.swapaxes(volume, 1,2)
midpoint = arr.shape[2] // 2
plt.title('Z {}'.format(i))
img = arr[:,:,midpoint]
#img = np.rot90(img, 1, axes=(1,0))
#img = np.flip(img, axis=1)
plt.imshow(img, cmap="seismic")
plt.show()

In [ ]:
#arr = normalize16(volume)
arr = volume
#arr = np.swapaxes(volume, 0,2)
midpoint = arr.shape[2] // 2
limit = 8
start = midpoint - 10
end = midpoint + 10
for i in tqdm(range(start, end, 4)):
    plt.title('Z {}'.format(i))
    img = arr[:,:,i]
    #img = np.rot90(img, 1, axes=(1,0))
    #img = np.flip(img, axis=1)
    plt.imshow(img, cmap="seismic")
    plt.show()

In [ ]:
image_stack = np.zeros(volume.shape)
file_list = []

for i in tqdm(range(0, volume.shape[2], 1)):
    img = volume[:,:,i]
    img = np.rot90(img, 1, axes=(1,0))
    img = np.flip(img, axis=1)
    filepath = os.path.join(OUTPUT, str(i).zfill(3) + '.tif')
    cv2.imwrite(filepath, img)

In [ ]:
# The default atlas created by brainreg is isotropic 25um
#sagittal = volume
print(f'Orig volume shape={sagittal.shape}')
vm2 = np.swapaxes(volume, 0,2)
#sagittal = np.rot90(sagittal, axes=(1, 0))
#sagittal = np.flip(sagittal, axis=1)
print(f'new volume shape={vm2.shape}')

dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='um',
    scales=[20,20,20])
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.dimensions = dimensions
    s.layout = '4panel'
    s.layers['Princeton'] = neuroglancer.SegmentationLayer(
        source=neuroglancer.LocalVolume(
            data=vm2,
            dimensions=dimensions,
            voxel_offset=(0, 0, 0),
        ))

print(viewer)